In [1]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [2]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [4]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [5]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [6]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

,Patient Number,State Patient Number,Date Announced,Age Bracket,Gender,Detected City,Detected District,Detected State,Current Status,Notes,...,Source_2,Source_3,Backup Notes,,,,,,,
0,1,KL-TS-P1,30/01/2020,20,F,Thrissur,Thrissur,Kerala,Recovered,Travelled from Wuhan,...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,,None
1,2,KL-AL-P1,02/02/2020,,,Alappuzha,Alappuzha,Kerala,Recovered,Travelled from Wuhan,...,https://weather.com/en-IN/india/news/news/2020...,,Student from Wuhan,,,,,,,
2,3,KL-KS-P1,03/02/2020,,,Kasaragod,Kasaragod,Kerala,Recovered,Travelled from Wuhan,...,https://twitter.com/ANI/status/122422148580539...,https://weather.com/en-IN/india/news/news/2020...,Student from Wuhan,,,,,,,
3,4,DL-P1,02/03/2020,45,M,East Delhi (Mayur Vihar),East Delhi,Delhi,Recovered,"Travelled from Austria, Italy",...,https://economictimes.indiatimes.com/news/poli...,,Travel history to Italy and Austria,,,,,,,
4,5,TS-P1,02/03/2020,24,M,Hyderabad,Hyderabad,Telangana,Recovered,,...,https://www.indiatoday.in/india/story/coronavi...,https://www.thehindu.com/news/national/coronav...,"Travel history to Dubai, Singapore contact",,,,,,,


# Data cleaning and transformations

In [7]:
# shape of dataframe
# ==================

p_df.shape

(2072, 24)

In [8]:
# columns
# =======

p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Status Change Date', 'Source_1', 'Source_2', 'Source_3',
       'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [9]:
# selecting important columns only
# ================================

p_df = p_df[['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Status Change Date', 'Source_1', 'Source_2', 'Source_3',
       'Backup Notes']]

In [10]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

In [11]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(2072, 17)
Patient Number 	 767
State Patient Number 	 1783
Date Announced 	 873
Age Bracket 	 1536
Gender 	 1495
Detected City 	 1403
Detected District 	 1029
Detected State 	 873
Current Status 	 873
Notes 	 881
Contracted from which Patient (Suspected) 	 1823
Nationality 	 1656
Status Change Date 	 876
Source_1 	 879
Source_2 	 1486
Source_3 	 1931
Backup Notes 	 1712


In [12]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(2072, 17)


Patient Number                                767
State Patient Number                         1783
Date Announced                                873
Age Bracket                                  1536
Gender                                       1495
Detected City                                1403
Detected District                            1029
Detected State                                873
Current Status                                873
Notes                                         881
Contracted from which Patient (Suspected)    1823
Nationality                                  1656
Status Change Date                            876
Source_1                                      879
Source_2                                     1486
Source_3                                     1931
Backup Notes                                 1712
dtype: int64

In [13]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

print(p_df.shape)
p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)

(2072, 17)
(1199, 17)


In [14]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Status Change Date', 'Source_1', 'Source_2', 'Source_3',
       'Backup Notes'],
      dtype='object')

In [24]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,current_status,notes,suspected_contacted_patient,nationality,status_change_date,source_1,source_2,source_3,backup_notes
272,273,NaN,21/03/2020,NaN,NaN,Pune,Pune,Maharashtra,Hospitalized,Travelled from Ireland,NaN,India,21/03/2020,https://twitter.com/ANI/status/124122041989548...,NaN,NaN,Travel History to Ireland
522,523,TN-P14,24/03/2020,52,F,Purasaivakkam,Chennai,Tamil Nadu,Hospitalized,Travelled from USA via London on 17 March,NaN,NaN,24/03/2020,https://twitter.com/Vijayabaskarofl/status/124...,NaN,NaN,NaN
42,43,KL-KN-1,09/03/2020,3,M,Kannur,Kannur,Kerala,Recovered,Travelled from Italy on 07/03/2020 (Parents in...,NaN,India,09/03/2020,https://www.hindustantimes.com/india-news/todd...,https://www.facebook.com/kkshailaja/posts/2892...,NaN,Travel from Italy on 07/03/2020 (Parents in is...
715,716,NaN,26/03/2020,NaN,NaN,NaN,Wayanad,Kerala,Hospitalized,Travelled from Abhudhabi EY 254 on 22 March la...,NaN,NaN,26/03/2020,https://twitter.com/xpresskerala/status/124315...,https://www.facebook.com/1777873788890698/post...,NaN,NaN
379,380,NaN,22/03/2020,69,M,,Ernakulam,Kerala,Deceased,Travelled from Dubai,NaN,India,28/03/2020,https://english.manoramaonline.com/news/kerala...,https://www.facebook.com/233008147097016/posts...,https://www.facebook.com/dcekm/posts/969574620...,NaN


# Saving Data

In [21]:
p_df.sample(5)

,patient_number,state_patient_number,date_announced,age_bracket,gender,detected_city,detected_district,detected_state,current_status,notes,contracted_from_which_patient_(suspected),nationality,status_change_date,source_1,source_2,source_3,backup_notes
1014,1015,KA-P79,28/03/2020,21,M,Nanjangud,Mysuru,Karnataka,Hospitalized,All the patients were contacts of Patient-52 (...,P689,NaN,28/03/2020,Press Bulletin,https://twitter.com/TOIBengaluru/status/124394...,NaN,NaN
1041,1042,NaN,29/03/2020,NaN,NaN,NaN,Srinagar,Jammu and Kashmir,Hospitalized,Details awaited,NaN,NaN,29/03/2020,https://twitter.com/kansalrohit69/status/12441...,NaN,NaN,NaN
1075,1076,TN-P43,29/03/2020,29,F,NaN,Coimbatore,Tamil Nadu,Hospitalized,Contact of TN-P26,P653,Indian,29/03/2020,https://twitter.com/Vijayabaskarofl/status/124...,NaN,NaN,NaN
74,75,KL-KN-2,12/03/2020,NaN,NaN,Kannur,Kannur,Kerala,Recovered,"Travelled from Dubai, SG 54",NaN,India,23/03/2020,https://english.manoramaonline.com/news/kerala...,https://www.thenewsminute.com/article/two-more...,https://www.facebook.com/CollectorKKD/posts/25...,Returned from Dubai
510,511,NaN,24/03/2020,NaN,NaN,Jabalpur,Jabalpur,Madhya Pradesh,Hospitalized,"Contact Transmission,Details awaited",NaN,NaN,23/03/2020,https://timesofindia.indiatimes.com/city/bhopa...,NaN,NaN,NaN


In [22]:
p_df.to_csv('patients_data.csv', index=False)